In [0]:
#setup 
import numpy as np
import pandas as pd
from google.colab import drive
import json
drive.mount('gdrive')

!pip install summa
from summa.summarizer import summarize

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at gdrive
     |████████████████████████████████| 61kB 4.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/6a/09/68/e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa


In [0]:
#load your data with your own address
df = pd.read_csv('gdrive/My Drive/NLU project/interim_data/111044/val.csv')

In [0]:
with open('gdrive/My Drive/NLU project/interim_data/111044/val.json', 'w') as f:
    f.write(df.to_json(orient='records'))

In [0]:
with open('gdrive/My Drive/NLU project/interim_data/111044/val.json') as f:
    data = json.load(f)

In [0]:
#parse 
#edited to fit the json format
body_list = []
for entry in data:
  out = ''
  for sentence in entry['Body'].split('\n'):
    if len(sentence) > 52 and sentence[0].isalpha():
      out += sentence
  body_list.append(out)


In [0]:
#test whether the input for summa is correctly formatted
text = body_list[2]
summarize(text, words=15)

'The beneficiaries involved are both Enron North America Corp.\nand Enron Canada Corp.'

In [0]:
#test whether the input for tr4w is correctly formattex
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(10)

##TextRank4Keyword

In [0]:
#TextRank4Keyword implementation
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        self.out = []

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            self.out.append(key)
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight


# Rouge

Using this implementation: https://pypi.org/project/py-rouge/

In [0]:
#setup rouge metrics
!pip install py-rouge

    100% |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
import rouge
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# predicting for all data using summa
for entry in data:
  body = entry['Body']
  entry['Hypothesis'] = summarize(body, words=15)

In [0]:
# predicting for all data using rank
for entry in data:
  body = entry['Body']
  tr4w = TextRank4Keyword()
  tr4w.analyze(body, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
  tr4w.get_keywords(10)
  out = tr4w.out
  out_str = ''
  for i in out:
    out_str = out_str + i + ' '
  entry['Hypothesis'] = out_str

In [0]:
#saving results to your own address
keyword_result = pd.DataFrame({'hyps':hyps,'refs':refs})
keyword_result.to_csv('gdrive/My Drive/NLU project/interim_data/111044/key_res.csv')

In [0]:
summa_result = pd.DataFrame({'hyps':hyps,'refs':refs})
summa_result.to_csv('gdrive/My Drive/NLU project/interim_data/111044/sum_res.csv')

Rouge for multiple sentences

In [0]:
#put results to rouge 
hyps, refs = map(list, zip(*[[d['Hypothesis'], d['Subject']] for d in data]))

In [0]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg']: #, 'Best', 'Individual'
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

    all_hypothesis = hyps
    all_references = refs

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
### skipp printing for individual rouge
#                 for reference_id in range(nb_references):
#                     print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
#                     print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 13.06	R: 32.30	F1: 17.06
	rouge-2:	P:  1.83	R:  5.10	F1:  2.41
	rouge-3:	P:  0.39	R:  1.18	F1:  0.52
	rouge-4:	P:  0.12	R:  0.40	F1:  0.17
	rouge-l:	P: 14.47	R: 31.38	F1: 18.55
	rouge-w:	P: 10.50	R: 21.32	F1: 12.39

Evaluation with Best


ZeroDivisionError: ignored

In [0]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg']: #, 'Best', 'Individual'
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

    all_hypothesis = hyps
    all_references = refs

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
### skipp printing for individual rouge
#                 for reference_id in range(nb_references):
#                     print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
#                     print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P:  5.96	R: 23.86	F1:  8.86
	rouge-2:	P:  2.49	R: 10.44	F1:  3.68
	rouge-3:	P:  1.31	R:  5.09	F1:  1.90
	rouge-4:	P:  0.83	R:  3.00	F1:  1.18
	rouge-l:	P:  7.37	R: 23.91	F1: 10.63
	rouge-w:	P:  5.32	R: 16.94	F1:  7.23

Evaluation with Best


ZeroDivisionError: ignored

In [0]:
#install bleu
!pip install sacrebleu

  Stored in directory: /root/.cache/pip/wheels/40/f1/a9/4cce0ec602e8d195da27bb9b8f6708ec778fbafdbabb097fde
Successfully built sacrebleu


In [0]:
#read results
df = pd.read_csv('gdrive/My Drive/NLU project/interim_data/111044/sum_res.csv')

In [0]:
ref = list(df['refs'])

In [0]:
#get bleu score
import sacrebleu
bleu = sacrebleu.raw_corpus_bleu(hyps, [ref],.01).score
print(bleu)

1.7075477944994566
